In [8]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchtext
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

   Unnamed: 0        Id                                              Title  \
0           0  34552656             Java: Repeat Task Every Random Seconds   
1           1  34553034                  Why are Java Optionals immutable?   
2           2  34553174  Text Overlay Image with Darkened Opacity React...   
3           3  34553318         Why ternary operator in swift is so picky?   
4           4  34553755                 hide/show fab with scale animation   

                                                Body  \
0  <p>I'm already familiar with repeating tasks e...   
1  <p>I'd like to understand why Java 8 Optionals...   
2  <p>I am attempting to overlay a title over an ...   
3  <p>The question is very simple, but I just cou...   
4  <p>I'm using custom floatingactionmenu. I need...   

                                                Tags         CreationDate  \
0                                     <java><repeat>  2016-01-01 00:21:59   
1                                   <jav

In [2]:
class network(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_dim = #
        self.n_layers = #
        self.batch_size = #
        
        self.gru = nn.GRU(10,self.hidden_dim,self.n_layers)#dummy numbers, figure it later
        self.fc = nn.Linear(self.hidden_dim,3)
        

    def forward(self, embedded):
        print(embedded)
        print(embedded.shape)
        hidden = self.init_hidden()
        out, hidden = self.gru(embedded, hidden)
        print(out)
        print(out.shape)
        out = self.fc(out)
        return out
    
    def init_hidden(self):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, self.batch_size, self.hidden_dim).zero_().to(device)
        return hidden

In [3]:
def generate_batch(batch):#not sure if this is necessary anymore since im padding w/ different script
    batch_body = [t[0] for t in batch]
    batch_y = [t[1] for t in batch]

    lengths_x1 = torch.tensor([t.shape[0] for t in batch_body])
    b_x1 = pad_sequence(batch_body, batch_first=True, padding_value=0)
    batch_y = torch.tensor(batch_y)

    return b_x1, lengths_x1, batch_y


In [4]:
def train(model, dataLoader):
    acc = 0
    for i, data in enumerate(dataLoader):
        text = data[0]
        label = data[1]
        optimizer.zero_grad()
        output = model(text)
        loss = loss(output, label)
        loss.backward()
        optimizer.step()
        acc += (output.argmax(1) == label).sum().item()

    # Adjust the learning rate
    scheduler.step()
    acc = acc/len(dataLoader)
    print(f'Train Accuracy: {acc}')

In [5]:
class myDataset(Dataset):
    def __init__(self, csv, bodyNumpyList):
        df = pd.read_csv(csv)
        self.body = pickle.load(bodyNumpyList)#this is a list of numpy arrays that have 0 padding at the beginning
        self.label = df['Y'].values.tolist()

        self.length = len(df)
       
    def __len__(self):
        return self.length
    
    def __getitem__(self,index):
        print(self.body[index])
        toReturn = [torch.tensor(self.body[index]), torch.tensor(np.array(self.label[index]))]
        return toReturn

In [6]:
EPOCH = 2
model = network()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)
trainData = myDataset('train-vectorized.csv', 'bodyVectorized.pkl')#dummy value
trainDataLoader = DataLoader(trainData, batch_size=8, shuffle=True)
                  #collate_fn=generate_batch)
for _ in range(EPOCH):
    train(model, trainDataLoader)


[[-1.9250215  -1.4107087   0.5058539  ...  1.1847291   0.14490049
   2.5677652 ]
 [-2.091426   -0.4364375   1.186873   ...  0.8489724   1.4412378
  -1.5522091 ]
 [-0.37578505  1.2096751  -2.2589872  ...  1.4128717   2.7247384
  -1.9226729 ]
 ...
 [-1.3919092  -0.22787613 -0.11009254 ... -0.5401942   0.3617995
  -1.2168816 ]
 [-0.10700941 -0.67001855  0.30983776 ... -0.6325072   1.6156385
  -2.5331757 ]
 [-0.1978892   1.9330659  -0.24133888 ...  0.7318607   1.3906918
   1.3111575 ]]


TypeError: can't convert np.ndarray of type numpy.str_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
testX = pickle.load('')#dummy value
testY = []
for chunk in pd.read_csv('train-vectorized.csv', chunksize = 1000):
    testY = np.concatenate(testY, chunk['Y'])
#testData = TensorDataset(torch.from_numpy(trainX), torch.from_numpy(trainY))
testData  = myDataset('test-vectorized.csv', 'pklPath.pkl')#dummy values
testDataLoader = DataLoader(testData, batch_size=8, shuffle=True)
#                  collate_fn=generate_batch)
with torch.no_grad():
    acc = 0
    for data in testDataLoader:#[text, label]
        text = data[0]
        label = data[1]
        output = model(text)
        acc += (output.argmax(1) == label).sum().item()
    print(f'Testing accuracy: {acc/len(trainDataLoader)}')
    